# game_utils2 Demo

This notebook demonstrates the new functional interface for defining games and strategies.

In [ ]:
from game_utils2 import RPS, Kuhn, FBCP, LCP, NLCP
from game_utils2 import InfoSet, StrategyProfile, normal_form_game, sequential_game
import numpy as np

## Example 1: Rock Paper Scissors

Simple normal form game with discrete actions and no types.

In [ ]:
# Get the game
game = RPS()
print(f"Game: {game['name']}")
print(f"Type: {game['type']}")
print(f"Actions: {game['actions']}")

# Sample types and test payoff
t0, t1 = game['types']()
print(f"\nTypes: {t0}, {t1}")

payoff = game['payoff']('R', 'S', None, None)
print(f"Payoff(Rock, Scissors): {payoff}")

In [ ]:
# Create a strategy profile
# Player 0: Always play Rock
# Player 1: Uniform random

strategy_p0 = {
    InfoSet(player=0, type=None, history=""): np.array([1.0, 0.0, 0.0])  # 100% Rock
}

strategy_p1 = {
    InfoSet(player=1, type=None, history=""): np.array([1/3, 1/3, 1/3])  # Uniform
}

profile = StrategyProfile(game, [strategy_p0, strategy_p1])
profile.summary()

In [ ]:
# Sample actions from the strategy
info_set = InfoSet(player=0, type=None, history="")
actions = [profile.sample_action(0, info_set) for _ in range(10)]
print(f"Player 0 actions: {actions}")  # Should be all 'R'

# Get probability of specific action
prob = profile.get_action_prob(1, InfoSet(1, None, ""), 'P')
print(f"\nPlayer 1 probability of Paper: {prob}")

## Example 2: Kuhn Poker

Sequential game with discrete types (cards) and actions.

In [ ]:
# Get the game
game = Kuhn(n_cards=3)
print(f"Game: {game['name']}")
print(f"Type: {game['type']}")

# Sample a deal
card0, card1 = game['types']()
print(f"\nDealt cards: Player 0 = {card0}, Player 1 = {card1}")

# Check available actions
p0_actions = game['p0_actions'](card0)
print(f"Player 0 actions: {p0_actions}")

# Player 1's actions depend on Player 0's action
p1_actions_after_bet = game['p1_actions'](card1, 'bet')
p1_actions_after_check = game['p1_actions'](card1, 'check')
print(f"Player 1 actions after bet: {p1_actions_after_bet}")
print(f"Player 1 actions after check: {p1_actions_after_check}")

In [ ]:
# Create a simple strategy
# Player 0: Always bet with King, always check with Jack/Queen
# Player 1: Always fold Jack, always call King, 50-50 with Queen

strategy_p0 = {
    InfoSet(0, 0, ""): np.array([1.0, 0.0]),  # Jack: check
    InfoSet(0, 1, ""): np.array([1.0, 0.0]),  # Queen: check
    InfoSet(0, 2, ""): np.array([0.0, 1.0]),  # King: bet
}

strategy_p1 = {
    InfoSet(1, 0, "bet"): np.array([1.0, 0.0]),    # Jack vs bet: fold
    InfoSet(1, 1, "bet"): np.array([0.5, 0.5]),    # Queen vs bet: 50-50
    InfoSet(1, 2, "bet"): np.array([0.0, 1.0]),    # King vs bet: call
    InfoSet(1, 0, "check"): np.array([1.0, 0.0]),  # Jack vs check: check
    InfoSet(1, 1, "check"): np.array([0.0, 1.0]),  # Queen vs check: bet
    InfoSet(1, 2, "check"): np.array([0.0, 1.0]),  # King vs check: bet
}

profile = StrategyProfile(game, [strategy_p0, strategy_p1])
profile.summary()

## Example 3: Limit Continuous Poker

Sequential game with continuous types (hand strengths) and discretized continuous actions (bet sizes).

In [ ]:
# Get the game
game = LCP(L=0.3, U=1.5, n_bet_sizes=50)
print(f"Game: {game['name']}")
print(f"Type: {game['type']}")

# Sample hand strengths
x, y = game['types']()
print(f"\nHand strengths: x = {x:.3f}, y = {y:.3f}")

# Check available bet sizes
p0_actions = game['p0_actions'](x)
print(f"\nPlayer 0 has {len(p0_actions)} actions (check + {len(p0_actions)-1} bet sizes)")
print(f"First 5 bet sizes: {[f'{a:.3f}' for a in p0_actions[:5]]}")
print(f"Last 5 bet sizes: {[f'{a:.3f}' for a in p0_actions[-5:]]}")

In [ ]:
# Test payoff function
# High hand beats low hand when called
payoff_high_called = game['payoff'](0.5, 'call', 0.8, 0.3)
print(f"Payoff(bet 0.5, call, high hand, low hand): {payoff_high_called}")

# Low hand folds
payoff_folded = game['payoff'](0.5, 'fold', 0.3, 0.8)
print(f"Payoff(bet 0.5, fold, low hand, high hand): {payoff_folded}")

# Check goes to showdown
payoff_check = game['payoff'](0, 'call', 0.8, 0.3)
print(f"Payoff(check, _, high hand, low hand): {payoff_check}")

## Example 4: Fixed Bet Continuous Poker

Normal form game with continuous types.

In [ ]:
# Get the game
game = FBCP(B=1.0)
print(f"Game: {game['name']}")
print(f"Type: {game['type']}")
print(f"Actions: {game['actions']}")

# Sample hand strengths
x, y = game['types']()
print(f"\nHand strengths: x = {x:.3f}, y = {y:.3f}")

# Test various outcomes
print("\nPayoffs:")
print(f"  (check, fold, x>y): {game['payoff']('check', 'fold', 0.8, 0.3)}")
print(f"  (check, call, x>y): {game['payoff']('check', 'call', 0.8, 0.3)}")
print(f"  (bet, fold, x<y): {game['payoff']('bet', 'fold', 0.3, 0.8)}")
print(f"  (bet, call, x>y): {game['payoff']('bet', 'call', 0.8, 0.3)}")
print(f"  (bet, call, x<y): {game['payoff']('bet', 'call', 0.3, 0.8)}")

## Example 5: Define Your Own Game

Use the builders to create custom games.

In [ ]:
# Custom normal form game: Matching Pennies
matching_pennies = normal_form_game(
    name="Matching Pennies",
    actions=[['H', 'T'], ['H', 'T']],
    payoff=lambda a0, a1, t0, t1: 1 if a0 == a1 else -1
)

print(f"Game: {matching_pennies['name']}")
print(f"Player 0 wins by matching: {matching_pennies['payoff']('H', 'H', None, None)}")
print(f"Player 0 loses by not matching: {matching_pennies['payoff']('H', 'T', None, None)}")

In [ ]:
# Custom sequential game: Simplified poker with 2 cards
import random

simple_poker = sequential_game(
    name="2-Card Simple Poker",
    types=lambda: tuple(random.sample([0, 1], 2)),  # Deal 2 cards
    p0_actions=lambda card: ['fold', 'bet'],
    p1_actions=lambda card, a0: ['fold', 'call'] if a0 == 'bet' else ['fold', 'bet'],
    payoff=lambda a0, a1, c0, c1: (
        -0.5 if a0 == 'fold' else
        0.5 if a1 == 'fold' else
        (1 if c0 > c1 else -1)  # Showdown
    )
)

print(f"Game: {simple_poker['name']}")
c0, c1 = simple_poker['types']()
print(f"Dealt: {c0}, {c1}")
print(f"Actions: {simple_poker['p0_actions'](c0)}")

## Next Steps

With these core components in place, we can now add:

1. **CFR Solver** - `solve(game, method='cfr', iterations=100000)`
2. **Best Response** - `best_response(game, opponent_strategy, responder=0)`
3. **Expected Payoff** - `profile.expected_payoff(player=0)`
4. **Visualization** - `profile.heatmap()`, `plot_convergence()`
5. **Analysis** - `compute_exploitability()`, `strategy_distance()`